In [1]:
import pandas as pd
import numpy as np
import itertools
from tqdm.notebook import tqdm
from _constants import *

In [2]:
def Radiation(tt, lat, J, P, specie): 
    """Radiation.
    
    
    
    Parameters
    ----------
    tt: array of time in a day
    lat: latitude
    J: julian day
    P: //
    specie: land cover - albedo
    
    Returns
    ----------
    Q: diurnal radiation

    """
    alfa_0 = alfas[specie]
    d = ds[specie]  
    phi = np.deg2rad(lat)
    
    Q = []
    for i in range(len(tt)):
        theta = 2*np.pi*(tt[i] - t_noon)/(24)
        sin_delta = 0.39785*np.sin(np.deg2rad(278.97+0.9856*J+1.9165*np.sin(np.deg2rad(356.6+0.9856*J))))
        delta = np.arcsin(sin_delta)
        cos_psi = np.sin(phi)*sin_delta + np.cos(phi)*np.cos(delta)*np.cos(theta)
        psi = np.arccos(cos_psi)

        air_mass = np.sqrt((r*np.cos(psi))**2 + 2.*r+1.) - r*np.cos(psi)
        Tr = 0.7**(air_mass**0.678)
        if np.cos(psi) <= 0.0:
            R_dir = 0.0
            alfa1 = 0.0
            Q.append(R_dir)
        else:
            R_dir = R_s0*Tr
            alfa1 = alfa_0*(1.+d)/(1.+2.*d*cos_psi)

            R_dif = 0.1*R_dir
            Rs = R_dir +R_dif

            Rn = Rs*(1.-alfa1) 
            G = 0.1* Rn 
            Q.append(Rn-G)
    return np.asarray(Q)

In [3]:
def Albedo(tt, lat, J, specie):
    """Albedo.
    
    Fai una breve descrizione di cosa fa.
    
    Parameters
    ----------
    tt: array of time in a day
    lat: latitude
    J: julian day
    specie: land cover - albedo
    
    Returns
    ----------
    alfa: diurnal albedo

    """
    alfa_0 = alfas[specie]
    d = ds[specie]  
    
    phi = np.deg2rad(lat)

    alfa = []
    for i in range(len(tt)):
        theta = 2*np.pi*(tt[i] - t_noon)/(24)
        sin_delta = 0.39785*np.sin(np.deg2rad(278.97+0.9856*J+1.9165*np.sin(np.deg2rad(356.6+0.9856*J))))
        delta = np.arcsin(sin_delta)
        cos_psi = np.sin(phi)*sin_delta + np.cos(phi)*np.cos(delta)*np.cos(theta)
        psi = np.arccos(cos_psi)
        
        if np.cos(psi) <= 0.0:
            R_dir = 0.0
            alfa1 = 0.0
            alfa.append(alfa1)
        else:
            alfa1 = alfa_0*(1.+d)/(1.+2.*d*cos_psi)
            alfa.append(alfa1)
    return np.asarray(alfa)

In [4]:
# Create temporal information.
# We consider a generic year of 365 days starting from 1° December (February with 28 days).
# Create temporal range of 1 year with sampling of a half an hour.
HM = pd.date_range("00:00", "23:30", freq = "30min").strftime("%H:%M:%S")
# Define the days number of this year.
JJ = np.arange(334, 699, 1)
# Define the month corresponding to each day.
MM = [31*[12], 31*[1], 28*[2], 31*[3], 30*[4], 31*[5], 30*[6], 31*[7], 31*[8], 30*[9], 31*[10], 30*[11]]
MM = list(itertools.chain.from_iterable(MM))
# Define the season that corresponds to each season.
seasons = {12:1, 1:1, 2:1, 3:2, 4:2, 5:2, 6:3, 7:3, 8:3, 9:4, 10:4, 11:4}
# Define numeric sampling of half an hour.
tt = np.arange(0, 24, 0.5)
# Define latitudes.
laty = np.arange(0, 66, 1) 

# Define empty dataframe to use.
df = pd.DataFrame(columns = pd.Index(laty, name = "Latitude"), index = pd.MultiIndex.from_tuples(list(map(lambda x: sum(x, ()), pd.MultiIndex.from_product([zip(*[MM, JJ]), zip(HM)]))), names = ["Month", "Day", "Hour"]))
df["Season"] = df.index.get_level_values("Month").map(lambda x: seasons[x])
df = df.set_index("Season", drop = True, append = True).reorder_levels([3, 0, 1, 2], axis = 0)
df.head()

Latitude                    0    1    2    3    4    5    6    7    8    9   \
Season Month Day Hour                                                         
1      12    334 00:00:00  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 00:30:00  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 01:00:00  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 01:30:00  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 02:00:00  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

Latitude                   ...   56   57   58   59   60   61   62   63   64  \
Season Month Day Hour      ...                                                
1      12    334 00:00:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 00:30:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 01:00:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 01:30:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                 02:00:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

Latitude                    65  
Season Month Day Hour           
1      12    334 00:00:00  NaN  
                 00:30:00  NaN  
                 01:00:00  NaN  
                 01:30:00  NaN  
                 02:00:00  NaN  

[5 rows x 66 columns]

In [5]:
# Define function able to compute radiation or albedo for each latitude.
def compute_func(group, func, specie):
    day = group.name
    if func == "Radiation":
        # Apply radiation to each column (latitude).
        group = group.apply(lambda x: Radiation(tt, x.name, day, P0, specie))
    elif func == "Albedo":
        # Apply albedo to each column (latitude).
        group = group.apply(lambda x: Albedo(tt, x.name, day, specie))     
    return group

In [6]:
# Define function in order to average results over seasons for each latitude.
def mean_func(x):
    return x.groupby(axis = 0, level = "Hour").mean()

## Radiation

In [7]:
tqdm.pandas()
df = df.groupby(axis = 0, level = "Day").progress_apply(lambda x: compute_func(x, "Radiation", "Black"))
df.head()

  0%|          | 0/365 [00:00<?, ?it/s]

Latitude                    0    1    2    3    4    5    6    7    8    9   \
Season Month Day Hour                                                         
1      12    334 00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 00:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Latitude                   ...   56   57   58   59   60   61   62   63   64  \
Season Month Day Hour      ...                                                
1      12    334 00:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 00:30:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:30:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 02:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Latitude                    65  
Season Month Day Hour           
1      12    334 00:00:00  0.0  
                 00:30:00  0.0  
                 01:00:00  0.0  
                 01:30:00  0.0  
                 02:00:00  0.0  

[5 rows x 66 columns]

In [8]:
final_df = df.groupby(axis = 0, level = "Season").apply(mean_func).unstack(level = "Season")
final_df.head()

Latitude   0                   1                   2        ...   63       \
Season      1    2    3    4    1    2    3    4    1    2  ...    3    4   
Hour                                                        ...             
00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
00:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

Latitude   64                       65                       
Season      1    2         3    4    1    2          3    4  
Hour                                                         
00:00:00  0.0  0.0  0.000000  0.0  0.0  0.0   0.000000  0.0  
00:30:00  0.0  0.0  0.000000  0.0  0.0  0.0   0.000000  0.0  
01:00:00  0.0  0.0  0.000000  0.0  0.0  0.0   0.000000  0.0  
01:30:00  0.0  0.0  0.000000  0.0  0.0  0.0   2.964345  0.0  
02:00:00  0.0  0.0  7.829605  0.0  0.0  0.0  21.138382  0.0  

[5 rows x 264 columns]

## Albedo

### Grass

In [9]:
tqdm.pandas()
df_alfa_grass = pd.DataFrame(columns = df.columns, index = df.index)
df_alfa_grass = df_alfa_grass.groupby(axis = 0, level = "Day").progress_apply(lambda x: compute_func(x, "Albedo", "Grass"))
df_alfa_grass.head()

  0%|          | 0/365 [00:00<?, ?it/s]

Latitude                    0    1    2    3    4    5    6    7    8    9   \
Season Month Day Hour                                                         
1      12    334 00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 00:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Latitude                   ...   56   57   58   59   60   61   62   63   64  \
Season Month Day Hour      ...                                                
1      12    334 00:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 00:30:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:30:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 02:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Latitude                    65  
Season Month Day Hour           
1      12    334 00:00:00  0.0  
                 00:30:00  0.0  
                 01:00:00  0.0  
                 01:30:00  0.0  
                 02:00:00  0.0  

[5 rows x 66 columns]

In [10]:
final_df_alfa_grass = df_alfa_grass.groupby(axis = 0, level = "Season").apply(mean_func).unstack(level = "Season")
final_df_alfa_grass.head()

Latitude   0                   1                   2        ...   63       \
Season      1    2    3    4    1    2    3    4    1    2  ...    3    4   
Hour                                                        ...             
00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
00:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

Latitude   64                       65                      
Season      1    2         3    4    1    2         3    4  
Hour                                                        
00:00:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.000000  0.0  
00:30:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.000000  0.0  
01:00:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.000000  0.0  
01:30:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.036474  0.0  
02:00:00  0.0  0.0  0.075709  0.0  0.0  0.0  0.126191  0.0  

[5 rows x 264 columns]

### Forest

In [11]:
tqdm.pandas()
df_alfa_forest = pd.DataFrame(columns = df.columns, index = df.index)
df_alfa_forest = df_alfa_forest.groupby(axis = 0, level = "Day").progress_apply(lambda x: compute_func(x, "Albedo", "Forest"))
df_alfa_forest.head()

  0%|          | 0/365 [00:00<?, ?it/s]

Latitude                    0    1    2    3    4    5    6    7    8    9   \
Season Month Day Hour                                                         
1      12    334 00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 00:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Latitude                   ...   56   57   58   59   60   61   62   63   64  \
Season Month Day Hour      ...                                                
1      12    334 00:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 00:30:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 01:30:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                 02:00:00  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Latitude                    65  
Season Month Day Hour           
1      12    334 00:00:00  0.0  
                 00:30:00  0.0  
                 01:00:00  0.0  
                 01:30:00  0.0  
                 02:00:00  0.0  

[5 rows x 66 columns]

In [12]:
final_df_alfa_forest = df_alfa_forest.groupby(axis = 0, level = "Season").apply(mean_func).unstack(level = "Season")
final_df_alfa_forest.head()

Latitude   0                   1                   2        ...   63       \
Season      1    2    3    4    1    2    3    4    1    2  ...    3    4   
Hour                                                        ...             
00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
00:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

Latitude   64                       65                      
Season      1    2         3    4    1    2         3    4  
Hour                                                        
00:00:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.000000  0.0  
00:30:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.000000  0.0  
01:00:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.000000  0.0  
01:30:00  0.0  0.0  0.000000  0.0  0.0  0.0  0.021515  0.0  
02:00:00  0.0  0.0  0.044781  0.0  0.0  0.0  0.075082  0.0  

[5 rows x 264 columns]

In [13]:
# Save results.
datasets = [final_df, final_df_alfa_grass, final_df_alfa_forest]
sheet_names = ["Radiation", "Albedo_grass", "Albedo_forest"]

excel_filename = "./Data/RadiationAlbedo.xlsx"
with pd.ExcelWriter(excel_filename) as writer:
    for data, sheet in zip(datasets, sheet_names):
        data.to_excel(writer, sheet_name = sheet, index_label = "Hour")

In [14]:
# Example.
xls = pd.ExcelFile("./Data/RadiationAlbedo.xlsx")

Radiation = pd.read_excel(xls, "Radiation", index_col = 0, header = [0, 1])
Radiation.head()

Latitude   0                   1                   2        ...   63       \
Season      1    2    3    4    1    2    3    4    1    2  ...    3    4   
Hour                                                        ...             
00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
00:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
01:30:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

Latitude   64                       65                       
Season      1    2         3    4    1    2          3    4  
Hour                                                         
00:00:00  0.0  0.0  0.000000  0.0  0.0  0.0   0.000000  0.0  
00:30:00  0.0  0.0  0.000000  0.0  0.0  0.0   0.000000  0.0  
01:00:00  0.0  0.0  0.000000  0.0  0.0  0.0   0.000000  0.0  
01:30:00  0.0  0.0  0.000000  0.0  0.0  0.0   2.964345  0.0  
02:00:00  0.0  0.0  7.829605  0.0  0.0  0.0  21.138382  0.0  

[5 rows x 264 columns]